In [34]:
#!python -m pip install matplotlib

In [35]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [36]:
import pandas as pd
import sklearn
import numpy as np
import nltk
#nltk.download('all')
pd.set_option('display.max_columns', 500)

In [37]:
VALID_DATA = pd.read_csv('data-1568374087583.csv') # reading the dataset

VALID_DATA = VALID_DATA.drop(axis=1, # getting rid of all the excess columns
                             columns=['id_agency', 'id_segment', 'id_subsegment', 'id_size_object', 'id_type_oper', 'id_wall_material', 'currency_advert', 'contact_person', 'coordinate_n', 'price_1sq_meter_uah', 'ln_price_1sq_meter_usd',
                                      'coordinate_e', 'coordinate_set_way', 'id_district', 'id_mikrodistrict', 'primary_agency', 'price_1sq_meter_usd',
                                     'id_metro', 'time_to_metro', 'land_area', 'realty_kod', 'id_re_advert_sq', 'is_valid', 'ln_price_usd',
                                     'time_exposition', 'distance_to_city', 'comfort', 'landscape', 'furnishing', 'price_uah', 'price_advert',
                                     'building_class', 'cadastral_number', 'buildings_on_land', 'wall_insulation', 'roof_type',
                                     'room_height', 'type_offer', 'type_obj', 'id_build_type_u', 'geom', 'geog', 'date_relevance'])

VALID_DATA = VALID_DATA.set_index('date_advert') # setting the index to be the dates of adverts 

In [38]:
# filling the nan values of a couple of columns
VALID_DATA['kitchen_area'] = VALID_DATA['kitchen_area'].fillna(value=np.average(VALID_DATA['kitchen_area'][VALID_DATA['kitchen_area'].notna()==True]))
VALID_DATA['living_area'] = VALID_DATA['living_area'].fillna(value=VALID_DATA['total_area'] - VALID_DATA['kitchen_area']) 
VALID_DATA['year_building'] = VALID_DATA['year_building'].fillna(value=np.average(VALID_DATA['year_building'][VALID_DATA['year_building'].notna()==True]))
# getting all the data that has street values that are not nan
VALID_DATA = VALID_DATA[VALID_DATA['street'].notna() == True]

In [39]:
POLIGON_lvl_3 = VALID_DATA['id_poligon_level_3'] # saving important columns for later use
HOUSES = VALID_DATA['house']
AD_TEXT = VALID_DATA['advert_text']
AD_URL = VALID_DATA['advert_url']
VALID_DATA = VALID_DATA.dropna(axis=1) # dropping all columns that have nan values

In [40]:
VALID_DATA['poligon_level_3'] = POLIGON_lvl_3 # returning saved columns
VALID_DATA['house'] = HOUSES
VALID_DATA['advert_text'] = AD_TEXT
VALID_DATA['advert_url'] = AD_URL
VALID_DATA['year_building'] = VALID_DATA['year_building'].astype(int) # casting the year building to int

In [41]:
VALID_DATA = VALID_DATA.drop(axis=1, columns=['id_poligon_level_1']) # removing the unnecessary column
VALID_DATA.head()

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,advert_url,street,year_building,poligon_level_3,house,advert_text
date_advert,,,,,,,,,,,,
2019-07-03,2,54800.0,1,52.0,20.000000,22.000000,https://www.lun.ua/uk/a/359198797,ВУЛ. ВЕРНАДСЬКОГО,1997,2818004.0,33,ЖК «Новодворянский» Премиум-класс расположен в...
2019-07-04,1,22542.9,1,39.0,1.000000,1.000000,https://www.lun.ua/uk/a/363863187,ПРОСП. МИРУ,1997,2818005.0,25,Продам 1-к квартиру в уникальном жилом комплек...
2019-06-24,1,35000.0,3,56.0,42.861938,13.138062,https://www.lun.ua/uk/a/363398034,ПРОСП. СЛОБОЖАНСЬКИЙ,1997,2818002.0,6А,Продам 3к начало пр. Правды. 2+1 чистое жилое....
2019-07-04,1,16500.0,2,45.0,29.000000,6.000000,https://www.lun.ua/uk/a/364018938,ВУЛ. КАРУНИ,1977,2818002.0,69,Отличная 2комнатная 43\29\6 по ул. Каруны на 5...
2019-07-04,1,24000.0,2,40.0,25.000000,7.000000,https://www.lun.ua/uk/a/364618985,ПРОСП. СЛОБОЖАНСЬКИЙ,1997,2818002.0,73,Продам 2 км кв на пр.Слобожанский район киноте...


In [42]:
VALID_DATA = VALID_DATA.dropna(axis=0) # dropping rows that still have nan values
#VALID_DATA.shape

VALID_DATA['poligon_level_1'] = VALID_DATA['poligon_level_3'].apply(lambda x: int(str(x)[:2])) # returning poligon columns
VALID_DATA['poligon_level_2'] = VALID_DATA['poligon_level_3'].apply(lambda x: int(str(x)[2:4]))
VALID_DATA['poligon_level_3'] = VALID_DATA['poligon_level_3'].apply(lambda x: int(str(x)[4:7]))
VALID_DATA.head()

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,advert_url,street,year_building,poligon_level_3,house,advert_text,poligon_level_1,poligon_level_2
date_advert,,,,,,,,,,,,,,
2019-07-03,2,54800.0,1,52.0,20.000000,22.000000,https://www.lun.ua/uk/a/359198797,ВУЛ. ВЕРНАДСЬКОГО,1997,4,33,ЖК «Новодворянский» Премиум-класс расположен в...,28,18
2019-07-04,1,22542.9,1,39.0,1.000000,1.000000,https://www.lun.ua/uk/a/363863187,ПРОСП. МИРУ,1997,5,25,Продам 1-к квартиру в уникальном жилом комплек...,28,18
2019-06-24,1,35000.0,3,56.0,42.861938,13.138062,https://www.lun.ua/uk/a/363398034,ПРОСП. СЛОБОЖАНСЬКИЙ,1997,2,6А,Продам 3к начало пр. Правды. 2+1 чистое жилое....,28,18
2019-07-04,1,16500.0,2,45.0,29.000000,6.000000,https://www.lun.ua/uk/a/364018938,ВУЛ. КАРУНИ,1977,2,69,Отличная 2комнатная 43\29\6 по ул. Каруны на 5...,28,18
2019-07-04,1,24000.0,2,40.0,25.000000,7.000000,https://www.lun.ua/uk/a/364618985,ПРОСП. СЛОБОЖАНСЬКИЙ,1997,2,73,Продам 2 км кв на пр.Слобожанский район киноте...,28,18


In [43]:
def CheckHouse(house): # removes letters from house numbers
    try:
        int(house)
        return int(house)
    except:
        ls = list(house)
        new_house = ''
        for x in ls:
            if str(x).isdigit():
                new_house += x
            else:
                break
        if not new_house:
            return 0
        else:
            return new_house

VALID_DATA['house'] = VALID_DATA['house'].apply(CheckHouse)

from sklearn.preprocessing import LabelEncoder # enconding street names to numeric values
labeler_streets = LabelEncoder().fit(VALID_DATA['street']) 
VALID_DATA['street'] = labeler_streets.transform(VALID_DATA['street'])

labeler_streets = LabelEncoder().fit(VALID_DATA['poligon_level_3']) 
VALID_DATA['poligon_level_3'] = labeler_streets.transform(VALID_DATA['poligon_level_3'])

labeler_streets = LabelEncoder().fit(VALID_DATA['poligon_level_1']) 
VALID_DATA['poligon_level_1'] = labeler_streets.transform(VALID_DATA['poligon_level_1'])

labeler_streets = LabelEncoder().fit(VALID_DATA['poligon_level_2']) 
VALID_DATA['poligon_level_2'] = labeler_streets.transform(VALID_DATA['poligon_level_2'])

CLEAN_VALID_DATA = VALID_DATA.copy()
CLEAN_VALID_DATA.head()

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,advert_url,street,year_building,poligon_level_3,house,advert_text,poligon_level_1,poligon_level_2
date_advert,,,,,,,,,,,,,,
2019-07-03,2,54800.0,1,52.0,20.000000,22.000000,https://www.lun.ua/uk/a/359198797,428,1997,3,33,ЖК «Новодворянский» Премиум-класс расположен в...,2,13
2019-07-04,1,22542.9,1,39.0,1.000000,1.000000,https://www.lun.ua/uk/a/363863187,2406,1997,4,25,Продам 1-к квартиру в уникальном жилом комплек...,2,13
2019-06-24,1,35000.0,3,56.0,42.861938,13.138062,https://www.lun.ua/uk/a/363398034,2440,1997,1,6,Продам 3к начало пр. Правды. 2+1 чистое жилое....,2,13
2019-07-04,1,16500.0,2,45.0,29.000000,6.000000,https://www.lun.ua/uk/a/364018938,841,1977,1,69,Отличная 2комнатная 43\29\6 по ул. Каруны на 5...,2,13
2019-07-04,1,24000.0,2,40.0,25.000000,7.000000,https://www.lun.ua/uk/a/364618985,2440,1997,1,73,Продам 2 км кв на пр.Слобожанский район киноте...,2,13


In [44]:
CORRELATIONS = CLEAN_VALID_DATA.corr('pearson') # getting correlation values for all the columns
CORRELATIONS = CORRELATIONS.drop(['poligon_level_1', 'poligon_level_2', 'poligon_level_3'], axis=1)  
CORRELATIONS.drop(['poligon_level_1', 'poligon_level_2', 'poligon_level_3'], axis=0).style.background_gradient(cmap='coolwarm') # applying a gradient

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,street,year_building
id_class_subsegment,1.000000,0.252455,-0.015222,0.329993,0.221691,0.404055,-0.052068,0.395319
price_usd,0.252455,1.000000,0.323118,0.744211,0.674287,0.365739,-0.071191,0.054700
qt_room,-0.015222,0.323118,1.000000,0.585741,0.572573,0.190310,-0.003484,-0.060085
total_area,0.329993,0.744211,0.585741,1.000000,0.883822,0.472828,-0.040946,0.089578
living_area,0.221691,0.674287,0.572573,0.883822,1.000000,0.252036,-0.032831,0.037477
kitchen_area,0.404055,0.365739,0.190310,0.472828,0.252036,1.000000,-0.054539,0.195216
street,-0.052068,-0.071191,-0.003484,-0.040946,-0.032831,-0.054539,1.000000,0.011228
year_building,0.395319,0.054700,-0.060085,0.089578,0.037477,0.195216,0.011228,1.000000


In [45]:
def color_map(val):
    if val < 0:
        return 'color: blue'
    elif val > .35 :
        return 'color: red'
    else:
        return 'color: white'

COLORED_CORRELATIONS = CORRELATIONS.drop(['poligon_level_1', 'poligon_level_2', 'poligon_level_3'], axis=0).style.applymap(color_map)
COLORED_CORRELATIONS # gradient that shows corr values to evaluate

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,street,year_building
id_class_subsegment,1.000000,0.252455,-0.015222,0.329993,0.221691,0.404055,-0.052068,0.395319
price_usd,0.252455,1.000000,0.323118,0.744211,0.674287,0.365739,-0.071191,0.054700
qt_room,-0.015222,0.323118,1.000000,0.585741,0.572573,0.190310,-0.003484,-0.060085
total_area,0.329993,0.744211,0.585741,1.000000,0.883822,0.472828,-0.040946,0.089578
living_area,0.221691,0.674287,0.572573,0.883822,1.000000,0.252036,-0.032831,0.037477
kitchen_area,0.404055,0.365739,0.190310,0.472828,0.252036,1.000000,-0.054539,0.195216
street,-0.052068,-0.071191,-0.003484,-0.040946,-0.032831,-0.054539,1.000000,0.011228
year_building,0.395319,0.054700,-0.060085,0.089578,0.037477,0.195216,0.011228,1.000000


In [46]:
CLUSTERING_DF = CLEAN_VALID_DATA.drop(columns=['advert_url'])
AD_TEXT = CLEAN_VALID_DATA['advert_text']
AD_URL = CLEAN_VALID_DATA['advert_url']
CLUSTERING_DF.head()

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,street,year_building,poligon_level_3,house,advert_text,poligon_level_1,poligon_level_2
date_advert,,,,,,,,,,,,,
2019-07-03,2,54800.0,1,52.0,20.000000,22.000000,428,1997,3,33,ЖК «Новодворянский» Премиум-класс расположен в...,2,13
2019-07-04,1,22542.9,1,39.0,1.000000,1.000000,2406,1997,4,25,Продам 1-к квартиру в уникальном жилом комплек...,2,13
2019-06-24,1,35000.0,3,56.0,42.861938,13.138062,2440,1997,1,6,Продам 3к начало пр. Правды. 2+1 чистое жилое....,2,13
2019-07-04,1,16500.0,2,45.0,29.000000,6.000000,841,1977,1,69,Отличная 2комнатная 43\29\6 по ул. Каруны на 5...,2,13
2019-07-04,1,24000.0,2,40.0,25.000000,7.000000,2440,1997,1,73,Продам 2 км кв на пр.Слобожанский район киноте...,2,13


# CLUSTERIZATION 
## K-means

In [47]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

class_1 = CLUSTERING_DF[CLUSTERING_DF['id_class_subsegment'] == 1].drop('advert_text', axis=1)
class_2 = CLUSTERING_DF[CLUSTERING_DF['id_class_subsegment'] == 2].drop('advert_text', axis=1)

In [48]:
minmax = MinMaxScaler().fit(class_1)
kmeans = KMeans(n_clusters=2, random_state=0).fit(minmax.transform(class_1)) # creating 4 clusters of apartment ads

In [49]:
kmeans_df = class_1.copy()
kmeans_df['targets'] = kmeans.predict(minmax.transform(class_1))
kmeans_df.head()

,id_class_subsegment,price_usd,qt_room,total_area,living_area,kitchen_area,street,year_building,poligon_level_3,house,poligon_level_1,poligon_level_2,targets
date_advert,,,,,,,,,,,,,
2019-07-04,1,22542.9,1,39.0,1.000000,1.000000,2406,1997,4,25,2,13,1
2019-06-24,1,35000.0,3,56.0,42.861938,13.138062,2440,1997,1,6,2,13,1
2019-07-04,1,16500.0,2,45.0,29.000000,6.000000,841,1977,1,69,2,13,1
2019-07-04,1,24000.0,2,40.0,25.000000,7.000000,2440,1997,1,73,2,13,1
2019-07-03,1,21000.0,2,44.0,37.000000,7.000000,462,1997,7,3,2,13,1


## Language processing and classification

In [50]:
kmeans_df[kmeans_df['targets']==1]['year_building'].mean()

1995.302102102102

In [51]:
kmeans_df[kmeans_df['targets']==0]['year_building'].mean()

1978.0170448131844

In [52]:
NLP = CLUSTERING_DF[['advert_text', 'id_class_subsegment']]
NLP

,advert_text,id_class_subsegment
date_advert,,
2019-07-03,ЖК «Новодворянский» Премиум-класс расположен в...,2
2019-07-04,Продам 1-к квартиру в уникальном жилом комплек...,1
2019-06-24,Продам 3к начало пр. Правды. 2+1 чистое жилое....,1
2019-07-04,Отличная 2комнатная 43\29\6 по ул. Каруны на 5...,1
2019-07-04,Продам 2 км кв на пр.Слобожанский район киноте...,1
...,...,...
2019-07-05,"Лукьяновка, Дмитриевская, 82, 4\20, 103\62\13 ...",2
2019-07-02,Просторная двусторонняя квартира в историческо...,2
2019-06-28,"От собственника, риэлторам не беспокоить. По в...",2


In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(NLP['advert_text'], NLP['id_class_subsegment'], train_size=.75)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
X_train_counts = CV.fit_transform(X_train)
X_train_counts

<41139x65250 sparse matrix of type '<class 'numpy.int64'>'
	with 2382640 stored elements in Compressed Sparse Row format>

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_counts)
X_train_tfidf

<41139x65250 sparse matrix of type '<class 'numpy.float64'>'
	with 2382640 stored elements in Compressed Sparse Row format>

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

In [70]:
classifier1 = Pipeline([('cv', CountVectorizer(stop_words='english')),
                       ('tfidf', TfidfTransformer(use_idf=True)),
                       ('forest', RandomForestClassifier())])

classifier1.fit(X_train, y_train)

predictions_forest = classifier1.predict(X_test)

In [58]:
classifier2 = Pipeline([('cv', CountVectorizer(ngram_range = (2, 2), stop_words='english')),
                       ('tfidf', TfidfTransformer()),
                       ('multinb', MultinomialNB(alpha=.1))])

classifier2.fit(X_train, y_train)

predictions_nb = classifier2.predict(X_test)

In [59]:
roc_auc_score(y_test, predictions_nb)

0.8061380599655031

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'cv__ngram_range': [(1, 1), (1, 2), (2, 3)], # (1,1) - ngram_range best choice
              # english - best stop words
              'forest__n_estimators': (100, 200, 300),
             }

gs_clf = GridSearchCV(classifier1, parameters, n_jobs=3, cv=2)
gs_clf = gs_clf.fit(X_train, y_train)

In [ ]:
gs_clf.best_score_

In [ ]:
gs_clf.best_params_